#### Construction de la base d'apprentissage

In [ ]:
# Pour construire la base d'apprentissage seulement
#from Bio import SeqIO
#import RNA

In [1]:
import numpy as np
import pandas as pd
import csv
from random import shuffle

In [2]:
def transformer_sequence(sequence,long):
    nucleotides = np.zeros((4,int(max(long,25))))
    cnt = 0
    for lettre in sequence:
        if lettre=='a':
            nucleotides[0,cnt] = 1
            cnt = cnt+1
        elif lettre=='c':
            nucleotides[1,cnt] = 1
            cnt = cnt+1
        elif lettre=='g':
            nucleotides[2,cnt] = 1
            cnt = cnt+1
        else:
            nucleotides[3,cnt] = 1
            cnt = cnt+1
    return nucleotides

In [3]:
def transformer_structure(structure):
    long = len(structure)
    a = np.zeros(25)
    a[:long] = structure
    return a

In [4]:
def traduire(seq,strand):
    nucleotides = ''
    if strand:
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'a'
            elif lettre=='C':
                nucleotides=nucleotides+'c'
            elif lettre=='G':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    else:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='A':
                nucleotides=nucleotides+'u'
            elif lettre=='C':
                nucleotides=nucleotides+'g'
            elif lettre=='G':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    return nucleotides

### Génération des bdd (ne pas exécuter)

In [4]:
parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")
dict_fasta = dict([(seq.id, seq) for seq in parser])

In [91]:
# base de données rise_human_targeted
bdd = pd.read_csv("rise_human_targeted.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='miRNA')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_sansM))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    varrbdd[i,1] = int(varrbdd[i,1])-1
    varrbdd[i,2] = int(varrbdd[i,2])
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = milieu-50-1
    varrbdd[i,5] = milieu+50
bdd = varrbdd

11473


In [237]:
writer = csv.writer(open("genomes/positifs.csv", 'w',newline=''),delimiter="\t")
    
for i in range(len(bdd)):
    pos = bdd[i,:4]
    ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
    pos[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = [bdd[i,3],bdd[i,4],bdd[i,5]]
    seq = dict_fasta[ide][begin:end].format("fasta").split('\n')
    pos[1] = traduire(seq[1]+seq[2],bdd[i,9])
    w = len(pos[0])
    pos[2] = np.array(RNA.pfl_fold_up(pos[0],1,w,w))[1:,1]
    pos[3] = np.array(RNA.pfl_fold_up(pos[1],1,101,101))[1:,1]
    writer.writerow(pos)        


In [238]:
# base de données rise_human_targeted
bdd = pd.read_csv("rise_human_targeted.csv", sep = "\t",header=None)
arrbdd = np.array(bdd)[:16650]

# Ne garder que les interactions miRNA - mRNA
arg_mi = np.argwhere(arrbdd[:,14]=='miRNA')[:,0]
arg_m = np.argwhere(arrbdd[:,15]=='protein_coding')[:,0]
args = [k for k in arg_mi if k in arg_m]
args = np.array(args)
arrbdd = arrbdd[args]

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'chrM') & (arrbdd[:,3] != 'chrM'))
arrbdd = arrbdd[arg_sansM[:,0]]
#enlever les champs vides
arg_sansM = np.argwhere((arrbdd[:,1] != '.') & (arrbdd[:,2] != '.') & (arrbdd[:,4] != '.') & (arrbdd[:,5] != '.') & (arrbdd[:,0] != '.') & (arrbdd[:,3] != '.') & (arrbdd[:,8] != '.') & (arrbdd[:,9] != '.'))
print(len(arg_sansM))
varrbdd = arrbdd[arg_sansM[:,0]]


#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = True
    else:
        varrbdd[i,8] = False

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = True
    else:
        varrbdd[i,9] = False

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    varrbdd[i,1] = int(varrbdd[i,1])-1
    varrbdd[i,2] = int(varrbdd[i,2])
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = milieu-50-1
    varrbdd[i,5] = milieu+50
bdd = varrbdd

10024


In [239]:
writer = csv.writer(open("genomes/negatifs.csv", 'w',newline=''),delimiter="\t")

for i in range(len(bdd)): 
    neg = bdd[i,:4]
    ide, begin, end = [bdd[i,0],bdd[i,1],bdd[i,2]]
    neg[0] = traduire(dict_fasta[ide][begin:end].format("fasta").split('\n')[1],bdd[i,8])
    ide, begin, end = bdd[i,3], bdd[i,4], bdd[i,5]
    begin = begin+80
    end = end+80
    sequ = dict_fasta[ide][begin:end].format("fasta").split('\n')
    neg[1] = traduire(sequ[1]+sequ[2],bdd[i,9])
    w = len(neg[0])
    neg[2] = np.array(RNA.pfl_fold_up(neg[0],1,w,w))[1:,1]
    neg[3] = np.array(RNA.pfl_fold_up(neg[1],1,101,101))[1:,1]
    writer.writerow(neg)

### Récupération et vérification des bases de données (exécuter à partir d'ici pour l'apprentissage)

### Réseau de neurones basé sur iDeep

In [7]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D

/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [9]:
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
#from keras.optimizers import kl_divergence
from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse

from keras.layers import Merge

In [55]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+ fp)
    sensitivity = float(tp)/ (tp+fn)
    specificity = float(tn)/(tn + fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 


In [99]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

verif = pd.read_csv("genomes/negatifs.csv", sep = "\t",header=None)
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(4):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

for i in range(len(verif)):
    verif[i,2] = np.array(clean(verif[i,2]))
    verif[i,3] = np.array(clean(verif[i,3]))    

neg = verif

verif = pd.read_csv("genomes/positifs.csv", sep = "\t",header=None)  
verif = np.array(verif)
for i in range(len(verif)):
    for j in range(4):
        if len(verif[i,j]) <= 15:
            print(i,"erreur")
            

for i in range(len(verif)):
    verif[i,2] = np.array(clean(verif[i,2]))
    verif[i,3] = np.array(clean(verif[i,3]))

pos = verif
verif = []
bdd = np.concatenate((pos,neg))
pos = []
neg = []
labels = np.zeros((len(bdd),1))

bdd = np.concatenate((bdd,labels),axis=1)
for i in range(int(len(bdd)/2)):
    bdd[i,4]=1    
labels=[]
for i in range(len(bdd)):
    bdd[i,0] = transformer_sequence(bdd[i,0],len(bdd[i,0]))
    bdd[i,1] = transformer_sequence(bdd[i,1],101)
    bdd[i,2] = transformer_structure(bdd[i,2])

# shuffle pour mélanger positifs et négatifs

indices = np.zeros(len(bdd),dtype=int)
for i in range(int(len(bdd)/2)):
    indices[2*i] = int(i)
    indices[2*i+1] = int(i + int(len(bdd)/2))
bdd = bdd[indices]
indices = []

# shuffle total

indices = np.arange(len(bdd))
shuffle(indices)
bdd = bdd[indices]

for i in range(len(bdd)):
    bdd[i,0] = bdd[i,0].transpose()
    bdd[i,1] = bdd[i,1].transpose()

resh0 = np.zeros((20048,25,4))

for i in range(len(bdd)):
    resh0[i,:,:] = bdd[i,0]
    
resh1 = np.zeros((20048,101,4))

for i in range(len(bdd)):
    resh1[i,:,:] = bdd[i,1]

resh2 = np.zeros((20048,25))

for i in range(len(bdd)):
    resh2[i,:] = bdd[i,2]
    
resh3 = np.zeros((20048,101))

for i in range(len(bdd)):
    resh3[i,:] = bdd[i,3]

In [100]:
nb_train = 17000
nb_val = 500
nb_test = 2500

In [101]:
resh0_train = resh0[:nb_train]
resh0_val = resh0[nb_train:nb_train+nb_val]
resh1_train = resh1[:nb_train]
resh1_val = resh1[nb_train:nb_train+nb_val]
resh2_train = resh2[:nb_train]
resh2_val = resh2[nb_train:nb_train+nb_val]
resh3_train = resh3[:nb_train]
resh3_val = resh3[nb_train:nb_train+nb_val]
resh0_test = resh0[-nb_test:]
resh1_test = resh1[-nb_test:]
resh2_test = resh2[-nb_test:]
resh3_test = resh3[-nb_test:]
resh0=[]
resh1=[]
resh2=[]
resh3=[]
train = bdd[:nb_train]
valid = bdd[nb_train:nb_train+nb_val]
y = train[:,4]
y = keras.utils.np_utils.to_categorical(y,2)
val_y = valid[:,4]
val_y = keras.utils.np_utils.to_categorical(val_y,2)
train=[]
valid=[]
test = bdd[-nb_test:]
bdd = []

In [102]:
def merge_networks_train_predict():
    
    print('training', nb_train)
    
    micro_structure_hid = 64
    messenger_structure_hid = 128
    micro_seq_hid = 22
    messenger_seq_hid = 100
    
    micro_seq_train = resh0_train
    micro_seq_validation = resh0_val
    micro_seq_net =  get_cnn_network_microRNA()
    
    messenger_seq_train = resh1_train
    messenger_seq_validation = resh1_val
    messenger_seq_net = get_cnn_network_messengerRNA()
    
    micro_structure_train = resh2_train
    micro_structure_validation = resh2_val
    micro_structure_net = get_mlp_microRNA()
    
    messenger_structure_train = resh3_train
    messenger_structure_validation = resh3_val
    messenger_structure_net = get_mlp_messengerRNA()        
    
    #y, encoder = preprocess_labels(training_label)
    #val_y, encoder = preprocess_labels(validation_label, encoder = encoder)
       
    
    #model = Sequential()
    training_net=[]
    training = []
    validation = []
    total_hid = 0
    
    training_net.append(micro_seq_net)
    training.append(micro_seq_train)
    validation.append(micro_seq_validation)
    total_hid = total_hid + micro_seq_hid
    micro_seq_train = []
    micro_seq_validation = [] 
    
    training_net.append(messenger_seq_net)
    training.append(messenger_seq_train)
    validation.append(messenger_seq_validation)
    total_hid = total_hid + messenger_seq_hid
    messenger_seq_train = []
    messenger_seq_validation = []
    
    training_net.append(micro_structure_net)
    training.append(micro_structure_train)
    validation.append(micro_structure_validation)
    total_hid = total_hid + micro_structure_hid
    micro_structure_train = []
    micro_structure_validation = []
    
    training_net.append(messenger_structure_net)
    training.append(messenger_structure_train)
    validation.append(messenger_structure_validation)
    total_hid = total_hid + messenger_structure_hid
    messenger_structure_train = []
    messenger_structure_validation = []
    
    model = Sequential()
    model.add(Merge(training_net, mode='concat'))
 
    #model.add(Dense(total_hid, input_shape=(total_hid,)))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    print(total_hid)
    model.add(Dense(2, input_shape=(total_hid,)))
    model.add(Activation('softmax'))
    
    #sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    
    
    #checkpointer = ModelCheckpoint(filepath="models/bestmodel.hdf5", verbose=0, save_best_only=True)
    earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=0)
    #validation_data=(np.transpose(validmat['validxdata'],axes=(0,2,1)), validmat['validdata']), callbacks=[checkpointer,earlystopper]
    print('model training')
    model.fit(training, y, batch_size=10000, epochs=1000, verbose=0, validation_data=(validation, val_y), callbacks=[earlystopper])
    
    training = []
    validation = []
    
    # test
    true_y = test[:,4]
    
    print('predicting')
    testing = []
    testing.append(resh0_test)
    testing.append(resh1_test)
    testing.append(resh2_test)
    testing.append(resh3_test)
        
    predictions = model.predict_proba(testing)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true_y)
    perfs = calculate_performance(len(predictions), predictions, true_y)
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

    
    return model

In [103]:
m = merge_networks_train_predict()

training 17000
configure cnn network
configure cnn network


/media/DATA/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


314
model training
predicting
[0.44264212 0.5621759  0.5483066  ... 0.6002486  0.54963225 0.56503737]
[0. 1. 1. ... 1. 1. 1.] [0.0 0.0 1 ... 1 1 1]
acc :  0.5564
precision :  0.5346628679962013
sensitivity :  0.8972111553784861
specificity :  0.21285140562248997
MCC :  0.15103195995226454


##### 100 trains : 
acc :  0.5
precision :  0.5333333333333333
sensitivity :  0.7272727272727273
specificity :  0.2222222222222222
MCC :  -0.058025885318565944

##### 17000 trains
acc :  0.5964
precision :  0.6069017254313578
sensitivity :  0.625193199381762
specificity :  0.5655058043117744
MCC :  0.19100235122777048
##### 
acc :  0.5564
precision :  0.5346628679962013
sensitivity :  0.8972111553784861
specificity :  0.21285140562248997
MCC :  0.15103195995226454
(10000 batch et 1000 epochs)

In [14]:
def get_cnn_network_microRNA():    
    print('configure cnn network')
    nbfilter = 22

    model = Sequential()
    model.add(Conv1D(activation="relu", input_shape=(25, 4), filters=22, kernel_size=7, strides=1, padding="valid"))

    model.add(MaxPooling1D(pool_size=3))
    
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    
    model.add(Dense(nbfilter, activation='relu'))
    model.add(Dropout(0.2))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Dense(64))
 
    #model.fit(X_train, y_train)
    
    return model

In [15]:
def get_cnn_network_messengerRNA():    
    print('configure cnn network')
    nbfilter = 100

    model = Sequential()
    model.add(Conv1D(activation="relu", input_shape=(101, 4), filters=100, kernel_size=7, strides=1, padding="valid"))

    model.add(MaxPooling1D(pool_size=3))
    
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dropout(0.25))
    
    #model.add(Dense(nbfilter, activation='relu'))
    #model.add(Activation('relu'))
    #model.add(PReLU())
    #model.add(BatchNormalization(mode=2))
    #model.add(Dense(64))    
    
    #model.fit(X_train, y_train)
    
    return model

In [16]:
def cnn2D():
    nb_conv = 4
    nb_pool = 2
    model = Sequential()
    model.add(Conv2D(64, (nb_conv, nb_conv), padding='valid', input_shape=(1, 101,4),strides=(1,1)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
    model.add(Dropout(0.25))
    model.add(Flatten())

In [17]:
def get_mlp_microRNA(num_hidden = 64):
    model = Sequential()

    #model.add(Dense(num_hidden, input_dim=train.shape[1], activation='relu'))
    model.add(Dense(num_hidden, input_shape=(25,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(num_hidden, input_dim=num_hidden, activation='relu'))
    #model.add(Dense(num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(sec_num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    return model

In [18]:
def get_mlp_messengerRNA(num_hidden = 128):
    model = Sequential()

    #model.add(Dense(num_hidden, input_dim=train.shape[1], activation='relu'))
    model.add(Dense(num_hidden, input_shape=(101,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_hidden, input_dim=num_hidden, activation='relu'))
    #model.add(Dense(num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    model.add(Dense(sec_num_hidden, input_shape=(num_hidden,), activation='relu'))
    model.add(PReLU())
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(Dropout(0.5))
    '''
    return model

### ARCHIVES

### Récupération rapide des séquences à partir de chr_nb start end

In [ ]:
# download genome at ftp://ftp.sanger.ac.uk/pub/gencode/Gencode_human/release_24/GRCh38.primary_assembly.genome.fa.gz

parser = SeqIO.parse(open("genomes/GRCh38.genome.fa"),"fasta")

dict_fasta = dict([(seq.id, seq) for seq in parser])

ide, begin, end = ['chr17',8222876,8222895]

dict_fasta[ide][begin:end].format("fasta").split('\n')[1]

### A partir de là c'est l'ancienne méthode (à conserver au cas où, archive)

#### On  sauvegarde la base de données dans un fichier csv

In [28]:
browser = webdriver.Chrome()
import csv

writer = csv.writer(open("genomes/database.csv", 'w',newline=''),delimiter="\t")
for i,el in enumerate(varrbdd):
    row = transformer_database2(varrbdd[i])    
    writer.writerow(row)

On cherche les trous dans les données

In [7]:
bdd_5000_trouee = pd.read_csv("genomes/database.csv", sep = "\t",header = None)

In [8]:
bdd_5000_trouee = np.array(bdd_5000_trouee)
l = []
for i,el in enumerate(bdd_5000_trouee):
    if type(bdd_5000_trouee[i,0])==float or type(bdd_5000_trouee[i,1])==float:
        a=0
    elif len(bdd_5000_trouee[i,0])==0 or len(bdd_5000_trouee[i,0])==0:
        a=0
    else:
        l.append(i)
l = np.array(l)
len(l)

4983

In [23]:
bdd_miRNA_mRNA = bdd_5000_trouee[l]

In [26]:
bdd_miRNA_mRNA = nettoyer(bdd_miRNA_mRNA)

In [29]:
bdd_miRNA_mRNA = traduire_séquences(bdd_miRNA_mRNA)

In [20]:
bdd_apprentissage = construire_seq_et_structure_bdd(bdd_miRNA_mRNA[:15])

#### A partir de mainenant, on essaye d'obtenir les probabilités d'accesibilité de chaque nucléotide sur une séquence quelconque ARN par exemple 'ACGUUGUCACACGAU'

In [10]:
def traduire_test(seq,strand):
    nucleotides = ''
    if strand:
        string = ''
        for i in seq:
            string = i + string
        seq = string
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'u'
            elif lettre=='c':
                nucleotides=nucleotides+'g'
            elif lettre=='g':
                nucleotides=nucleotides+'c'
            else:
                nucleotides=nucleotides+'a'
    else:
        for lettre in seq:
            if lettre=='a':
                nucleotides=nucleotides+'a'
            elif lettre=='c':
                nucleotides=nucleotides+'c'
            elif lettre=='g':
                nucleotides=nucleotides+'g'
            else:
                nucleotides=nucleotides+'u'    
    return nucleotides

In [55]:
traduire_test('gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg',True)

'cacuacagcaagccuggcacgaccucuaaggcgguucgcggcaacguccgcacgucggcucguuggucuagggguaugauucucgcuucgggugcgagaggucccggguucaaaucccggacgagcccuccuuuaccuuuuacugagacaagagugucuuc'

In [ ]:
'gaagacactcttgtctcagtaaaaggtaaaggagggctcgtccgggatttgaacccgggacctctcgcacccgaagcgagaatcatacccctagaccaacgagccgacgtgcggacgttgccgcgaaccgccttagaggtcgtgccaggcttgctgtagtg'

In [12]:
bdd = pd.read_csv("rise_human_transcriptome (copie).csv", sep = "\t",header=None)

/media/DATA/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
# bdd convertie panda -> numpy array et seuls les 70 000 premiers RRIs retenus
arrbdd = np.array(bdd)[:70000]

#enlever 'chr' et formater les numéros de chromosomes
for i,el in enumerate(arrbdd[:,0]):
    arrbdd[i,0] = arrbdd[i,0][3:]

for i,el in enumerate(arrbdd[:,0]):
    if arrbdd[i,0] == 'X':
        arrbdd[i,0] = '23'
    if arrbdd[i,0] == 'Y':
        arrbdd[i,0] = '24'
    if arrbdd[i,0] == '1':
        arrbdd[i,0] = '01'
    if arrbdd[i,0] == '2':
        arrbdd[i,0] = '02'
    if arrbdd[i,0] == '3':
        arrbdd[i,0] = '03'
    if arrbdd[i,0] == '4':
        arrbdd[i,0] = '04'
    if arrbdd[i,0] == '5':
        arrbdd[i,0] = '05'
    if arrbdd[i,0] == '6':
        arrbdd[i,0] = '06'
    if arrbdd[i,0] == '7':
        arrbdd[i,0] = '07'
    if arrbdd[i,0] == '8':
        arrbdd[i,0] = '08'
    if arrbdd[i,0] == '9':
        arrbdd[i,0] = '09'

for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,3] = arrbdd[i,3][3:]

for i,el in enumerate(arrbdd[:,3]):
    if arrbdd[i,3] == 'X':
        arrbdd[i,3] = '23'
    elif arrbdd[i,3] == 'Y':
        arrbdd[i,3] = '24'  
    elif arrbdd[i,3] == '1':
        arrbdd[i,3] = '01'
    elif arrbdd[i,3] == '2':
        arrbdd[i,3] = '02'
    elif arrbdd[i,3] == '3':
        arrbdd[i,3] = '03'
    elif arrbdd[i,3] == '4':
        arrbdd[i,3] = '04'
    elif arrbdd[i,3] == '5':
        arrbdd[i,3] = '05'
    elif arrbdd[i,3] == '6':
        arrbdd[i,3] = '06'
    elif arrbdd[i,3] == '7':
        arrbdd[i,3] = '07'
    elif arrbdd[i,3] == '8':
        arrbdd[i,3] = '08'
    elif arrbdd[i,3] == '9':
        arrbdd[i,3] = '09'


#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'M') & (arrbdd[:,3] != 'M'))
varrbdd = arrbdd[arg_sansM[:,0]]

#changement strand
for i,el in enumerate(varrbdd[:,8]):
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = 'true'
    else:
        varrbdd[i,8] = 'false'

for i,el in enumerate(varrbdd[:,9]):
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = 'true'
    else:
        varrbdd[i,9] = 'false'


#int str
for i,el in enumerate(varrbdd[:,1]):
    if type(el) == int:
        varrbdd[i,1] = str(el)
for i,el in enumerate(varrbdd[:,2]):
    if type(el) == int:
        varrbdd[i,2] = str(el)
for i,el in enumerate(varrbdd[:,4]):
    if type(el) == int:
        varrbdd[i,4] = str(el)
for i,el in enumerate(varrbdd[:,5]):
    if type(el) == int:
        varrbdd[i,5] = str(el)

#génération position début et fin de séquence (nucléotides)

for i,el in enumerate(varrbdd[:,1]):
    milieu = int((int(varrbdd[i,1])+int(varrbdd[i,2]))/2)
    varrbdd[i,1] = str(milieu-50)
    varrbdd[i,2] = str(milieu+50)
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = str(milieu-50)
    varrbdd[i,5] = str(milieu+50)

# Génération URLs

#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,0]+"?report=genbank&from="+varrbdd[i,1]+"&to="+varrbdd[i,2]+"&strand="+varrbdd[i,8]+".html"
#for i,el in enumerate(varrbdd[:,6]):
 #   varrbdd[i,6] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,0]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,1]+'&to='+varrbdd[i,2]

#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,3]+"?report=genbank&from="+varrbdd[i,4]+"&to="+varrbdd[i,5]+"&strand="+varrbdd[i,9]+".html"
#for i,el in enumerate(varrbdd[:,7]):
 #   varrbdd[i,7] = 'https://www.ncbi.nlm.nih.gov/nuccore/NC_0000'+varrbdd[i,3]+'?report=fasta&log$=seqview&format=text&from='+varrbdd[i,4]+'&to='+varrbdd[i,5]

In [ ]:
def recuperer_sequence(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(4)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    browser.quit()
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("span",{'class':"ff_line"})
    rep = seq[0].text+seq[1].text+seq[2].text+seq[3].text
    rep = ''.join(rep.split())
    return rep

def recuperer_sequence2(url):
    browser.get(url)
    time.sleep(1.3)
    innerHTML = browser.execute_script("return document.body.innerHTML")
    soup = BeautifulSoup(innerHTML,"lxml")
    seq = soup.find_all("div",{'class':"seq gbff"})
    rep = seq[0].text   
    return rep

def transformer_database(bddrow):
    sequence=recuperer_sequence(bddrow[6])
    sequence=transformer_sequence(sequence,len(sequence))
    sequencebis=recuperer_sequence(bddrow[7])
    sequencebis=transformer_sequence(sequencebis,len(sequencebis))    
    bddrow = np.array([bddrow[18],bddrow[19],bddrow[20],bddrow[21],sequence,bddrow[0],bddrow[1],bddrow[2],bddrow[8],bddrow[10],bddrow[11],bddrow[14],bddrow[16],sequencebis,bddrow[3],bddrow[4],bddrow[5],bddrow[9],bddrow[12],bddrow[13],bddrow[15],bddrow[17]], dtype=object)
    return bddrow

def transformer_database2(bddrow):
    sequence=recuperer_sequence2(bddrow[6])
    sequencebis=recuperer_sequence2(bddrow[7])
    bddrow = np.array([sequence,sequencebis,bddrow[8],bddrow[9]], dtype=object)
    return bddrow

def construire_seq_et_structure_bdd(bdd):
    for i,el in enumerate(bdd):
        bdd[i,2] = np.array(RNA.pfl_fold_up(bdd[i,0],1,201,201))[1:,1]
        bdd[i,3] = np.array(RNA.pfl_fold_up(bdd[i,1],1,201,201))[1:,1]
        #bdd[i,0] = transformer_sequence(bdd[i,0],201)
        #bdd[i,1] = transformer_sequence(bdd[i,1],201)
    return bdd

def nettoyer(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=bdd[i,0].split('\n')[1]+bdd[i,0].split('\n')[2]+bdd[i,0].split('\n')[3]
        bdd[i,1]=bdd[i,1].split('\n')[1]+bdd[i,1].split('\n')[2]+bdd[i,1].split('\n')[3]
    return bdd

def traduire_séquences(bdd):
    for i,el in enumerate(bdd):
        bdd[i,0]=traduire(bdd[i,0],bdd[i,2])
        bdd[i,1]=traduire(bdd[i,1],bdd[i,3])
    return bdd